In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/10623/team project
!pwd

/content/drive/MyDrive/10623/team project
/content/drive/MyDrive/10623/team project


In [3]:
!ls

perplexity.ipynb  samples  samples.zip


In [5]:
import re

def html2raw(text):
  pattern = re.compile(r'<[^>]*>')
  cleaned_text = re.sub(pattern, '', text)

  return cleaned_text


In [6]:
import json

def json2str(json_file_path):
  data = None
  with open(json_file_path, 'r', encoding='utf-16') as file:
      data = json.load(file)

  title = data['title']['raw']
  abstract = html2raw(data['abstract']['raw'])

  desc = []
  for k in data['description']:
    if 'raw' in data['description'][k]:
      text = data['description'][k]['raw']
      desc.append(html2raw(text))
  description = "".join(desc)

  # claims = str(data['claims'])

  return title + "\n\n" + abstract + "\n\n" + description


In [7]:
import os

directory = "samples"
patents = []
for filename in os.listdir(directory):
  path = os.path.join(directory, filename)
  patents.append(json2str(path))

print(len(patents))


100


In [8]:
input_text = "\n\n".join(patents)

In [9]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

device = "cuda"
model_id = "openai-community/gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
# text = "bayern munich is the champion of the ucl league 2024"
encodings = tokenizer(input_text, return_tensors="pt")


Token indices sequence length is longer than the specified maximum sequence length for this model (1378596 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
print(encodings['input_ids'].shape)

torch.Size([1, 1378596])


In [16]:
import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 256
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())


100%|█████████▉| 5382/5386 [16:00<00:00,  5.60it/s]


In [17]:
print(ppl)


tensor(5.9578, device='cuda:0')
